# Construir la Red Neuronal

Las redes neuronales se componen de capas/módulos que realizan operaciones en datos.
El espacio de nombres [torch.nn](https://pytorch.org/docs/stable/nn.html) proporciona todos los bloques de construcción que necesitas para
construir tu propia red neuronal. Cada módulo en PyTorch es una subclase de [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
Una red neuronal es un módulo en sí mismo que consiste en otros módulos (capas). Esta estructura anidada permite
construir y gestionar arquitecturas complejas fácilmente.

En las siguientes secciones, construiremos una red neuronal para clasificar imágenes en el conjunto de datos FashionMNIST.

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Obtener Dispositivo para Entrenamiento

Queremos poder entrenar nuestro modelo en un [acelerador](https://pytorch.org/docs/stable/torch.html#accelerators)
como CUDA, MPS, MTIA, o XPU. Si el acelerador actual está disponible, lo usaremos. De lo contrario, usamos la CPU.

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Usando dispositivo {device}")

## Definir la Clase

Definimos nuestra red neuronal heredando de ``nn.Module``, e
inicializamos las capas de la red neuronal en ``__init__``. Cada subclase de ``nn.Module`` implementa
las operaciones en datos de entrada en el método ``forward``.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Creamos una instancia de ``NeuralNetwork``, la movemos al ``device``, e imprimimos
su estructura.

In [ ]:
model = NeuralNetwork().to(device)
print(model)

Para usar el modelo, le pasamos los datos de entrada. Esto ejecuta el ``forward`` del modelo,
junto con algunas [operaciones en segundo plano](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
¡No llames a ``model.forward()`` directamente!

Llamar al modelo con la entrada devuelve un tensor bidimensional con dim=0 correspondiente a cada salida de 10 valores predichos sin procesar para cada clase, y dim=1 correspondiente a los valores individuales de cada salida.
Obtenemos las probabilidades de predicción pasándolo a través de una instancia del módulo ``nn.Softmax``.

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

---

## Capas del Modelo

Desglosemos las capas en el modelo FashionMNIST. Para ilustrarlo,
tomaremos un minilote de muestra de 3 imágenes de tamaño 28x28 y veremos qué le sucede mientras
lo pasamos a través de la red.

In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

### nn.Flatten

Inicializamos la capa [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
para convertir cada imagen 2D de 28x28 en un arreglo contiguo de 784 valores de píxeles (
la dimensión del minilote (en dim=0) se mantiene).

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

### nn.Linear

La [capa lineal](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
es un módulo que aplica una transformación lineal en la entrada usando sus pesos y sesgos almacenados.

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

### nn.ReLU

Las activaciones no lineales son las que crean las asignaciones complejas entre las entradas y salidas del modelo.
Se aplican después de las transformaciones lineales para introducir *no linealidad*, ayudando a las redes neuronales
a aprender una amplia variedad de fenómenos.

En este modelo, usamos [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) entre nuestras
capas lineales, pero hay otras activaciones para introducir no linealidad en tu modelo.

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

### nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) es un contenedor ordenado
de módulos. Los datos se pasan a través de todos los módulos en el mismo orden en que se definen. Puedes usar
contenedores secuenciales para armar rápidamente una red como ``seq_modules``.

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits

### nn.Softmax

La última capa lineal de la red neuronal devuelve `logits` - valores sin procesar en [-∞, ∞] - que se pasan al
módulo [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html). Los logits se escalan a valores
[0, 1] que representan las probabilidades predichas del modelo para cada clase. El parámetro ``dim`` indica la dimensión a lo largo de
la cual los valores deben sumar 1.

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab*100

## Parámetros del Modelo

Muchas capas dentro de una red neuronal están *parametrizadas*, es decir, tienen pesos
y sesgos asociados que se optimizan durante el entrenamiento. Heredar de ``nn.Module`` automáticamente
rastrea todos los campos definidos dentro de tu objeto modelo, y hace que todos los parámetros
sean accesibles usando los métodos ``parameters()`` o ``named_parameters()`` de tu modelo.

En este ejemplo, iteramos sobre cada parámetro, e imprimimos su tamaño y una vista previa de sus valores.

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

---

## Lectura Adicional

- [torch.nn API](https://pytorch.org/docs/stable/nn.html)